In [1]:
import pandas as pd
import numpy as np
import os
from ete3 import NCBITaxa, PhyloTree
ncbi = NCBITaxa()
from collections import Counter
import gseapy as gp
from gseapy.plot import barplot, dotplot

In [2]:
table_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Tables\\'
fig_path='C:\\Users\\huangyan8\\Desktop\\work\\2024-09-03 TE_Evolution_Draft\\Figures\\'
samples=pd.read_excel(table_path+'Table S1\\Table S1.TE_Evo_352_Species.xlsx')
samples.shape

(352, 7)

### Enrichment of TE-associated Genes

In [4]:
family=pd.read_csv("E:\\Linux Work Project\\2023-01-14 Genome\\12.ncRNA\\family.txt",sep='\t',header=None,encoding_errors='ignore')
family_dic={}
class_dic={}
for i in range(family.shape[0]):
    family_dic[family.loc[i,0]]=family.loc[i,1]
    class_dic[family.loc[i,0]]=family.loc[i,3]

In [3]:
ncRNA_path='D:\\18.TE_Evolution\\06.Bed\\ncRNA_Bed\\'
ncRNA_sets={}
for i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+"_ncRNA.bed" in os.listdir(ncRNA_path):
        df=pd.read_csv(ncRNA_path+specie_id+"_ncRNA.bed",sep='\t')
        new=[]
        for n,n_df in df.groupby("ncRNA_name"):
            n_df=n_df.reset_index(drop=True).reset_index()
            n_df['name']=n_df['ncRNA_name']+"_"+specie_id+"_"+n_df['index'].apply(lambda x:str(x))
            new.append(n_df)
        df=pd.concat(new).reset_index(drop=True)
        for nf,nf_df in df.groupby("ncRNA_Family"):
            genes=nf_df["name"].tolist()
            for gene in genes:
                if nf in ncRNA_sets:
                    ncRNA_sets[nf].append(gene)
                else:
                    ncRNA_sets[nf]=[gene]

In [5]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part2.TE-associated_Genetic_Functions\\'
ncRNA_path='D:\\18.TE_Evolution\\04.TE\\TE_ncRNA_list\\Overall\\'
gene_list=[]
count=0
for  i in range(samples.shape[0]):
    specie_id=samples.loc[i,'Specie_ID']
    if specie_id+".TE_to_ncRNA.txt" in os.listdir(ncRNA_path):
        te_ncRNAs=[gene.strip() for gene in open(ncRNA_path+specie_id+".TE_to_ncRNA.txt",'r').readlines()]
        gene_list+=te_ncRNAs
gene_list=list(set(gene_list))
count=len(gene_list)
print('TE-associated ncRNAs',count)
if len(gene_list)>10:
    try:
        enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
            gene_sets=ncRNA_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
            outdir=save_path,#输出目录
            top_term=20,
            cutoff=0.01#pvalue阈值
                                             )
        result=enr.results
        result=result.sort_values("Adjusted P-value")
        result["Title"]=result['Term'].apply(lambda x:class_dic[x] if x in class_dic else '')
        print(result.shape)
        result.to_excel(save_path+"TE-associated_ncRNAs_Enrichment.xlsx",index=False)
    except:
        pass
else:
    print('Error: No TE-associated genes!')

TE-associated ncRNAs 588945


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


(322, 9)


In [6]:
result[result["Adjusted P-value"]<0.01].shape

(37, 9)

### Enrichment of TE Origing ncRNAs

In [36]:
ncRNA_sets={}
ncRNA_bed_path='D:\\18.TE_Evolution\\06.Bed\\ncRNA_Bed\\'
for i in range(samples.shape[0]):
    if i%100==0:
        print(i)
    specie_id=samples.loc[i,'Specie_ID']
    bg_file=specie_id+".TE_to_ncRNA.txt"
    bg_genes={}
    if bg_file in os.listdir(ncRNA_path):
        te_ncRNAs=[gene.strip() for gene in open(ncRNA_path+specie_id+".TE_to_ncRNA.txt",'r').readlines()]
        for gene in te_ncRNAs:
            bg_genes[gene]=1
    if specie_id+"_ncRNA.bed" in os.listdir(ncRNA_bed_path):
        df=pd.read_csv(ncRNA_bed_path+specie_id+"_ncRNA.bed",sep='\t')
        new=[]
        for n,n_df in df.groupby("ncRNA_name"):
            n_df=n_df.reset_index(drop=True).reset_index()
            n_df['name']=n_df['ncRNA_name']+"_"+specie_id+"_"+n_df['index'].apply(lambda x:str(x))
            new.append(n_df)
        df=pd.concat(new).reset_index(drop=True)
        for nf,nf_df in df.groupby("ncRNA_Family"):
            genes=nf_df["name"].tolist()
            te_genes=[]
            for gene in genes:
                if gene in bg_genes:
                    te_genes.append(gene)
            if nf in ncRNA_sets:
                ncRNA_sets[nf]+=te_genes
            else:
                ncRNA_sets[nf]=te_genes

0
100
200
300


In [37]:
gene_list_path='D:\\18.TE_Evolution\\04.TE\\TE_ncRNA_list\\Super_Family\\'
families=list(set([f.split(".TE_to_ncRNA")[0][:-5] for f in os.listdir(gene_list_path)]))
families

['Helitron',
 'LTR Unknown',
 'Mutator',
 'Unknown',
 'PIF-Harbinger',
 'Copia',
 'hAT',
 'Other',
 'Gypsy',
 'CACTA',
 'Tc1_Mariner']

In [38]:
save_path='D:\\18.TE_Evolution\\12.Analysis\\2024-08-16 TE Evolution\\Part1.Family_Enrichment\\'
for family in families:
    gene_list=[]
    for  i in range(samples.shape[0]):
        specie_id=samples.loc[i,'Specie_ID']
        g_file=family+"_"+specie_id+'.TE_to_ncRNA.txt'#".TE_DupGenes.txt"
        if g_file in os.listdir(gene_list_path):
            genes=[gene.strip() for gene in open(gene_list_path+g_file,'r').readlines()]
            gene_list+=genes
    print(family,len(gene_list))
    if len(gene_list)>10:
        try:
            enr = gp.enrichr(gene_list=gene_list,#所需查询gene_list，可以是一个列表，也可为文件（一列，每行一个基因）
                            gene_sets=ncRNA_sets,#gene set library，多个相关的gene set 。如所有GO term组成一个gene set library.
                            outdir=save_path,#输出目录
                            top_term=20,
                            cutoff=0.01#pvalue阈值
                                             )
            result=enr.results
            result=result.sort_values("Adjusted P-value")
            result["Title"]=result['Term'].apply(lambda x:class_dic[x] if x in class_dic else '')
            print(family,result.shape)
            result.to_excel(save_path+family+"_ncRNAs_TE_Origing_Enrichment.xlsx",index=False)
        except:
            pass
    else:
        print(family)

Helitron 84432


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Helitron (194, 9)
LTR Unknown 43932


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


LTR Unknown (108, 9)
Mutator 109244


...

Gypsy (91, 9)
CACTA 50044


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


CACTA (138, 9)
Tc1_Mariner 194472


C:\Users\huangyan8\Anaconda3\lib\site-packages\gseapy\enrichr.py:556: RuntimeWarning: divide by zero encountered in log
  odict["Combined Score"] = -1 * log(pvals) * oddr


Tc1_Mariner (120, 9)
